##Exemplo Classificador de Flores - TensorFlow
 

(iris.jpg)

In [1]:
import tensorflow as tf
import pandas as pd

COLUMN_NAMES = [
        'SepalLength', 
        'SepalWidth',
        'PetalLength', 
        'PetalWidth', 
        'Species'
        ]

# Importando dataset de treinamento
training_dataset = pd.read_csv('iris_training.csv', names=COLUMN_NAMES, header=0)
train_x = training_dataset.iloc[:, 0:4]
train_y = training_dataset.iloc[:, 4]

# Importando dataset de teste
test_dataset = pd.read_csv('iris_test.csv', names=COLUMN_NAMES, header=0)
test_x = test_dataset.iloc[:, 0:4]
test_y = test_dataset.iloc[:, 4]


(iris_dataset.png)

In [2]:
# Agora, precisamos definir colunas de recursos, que irão ajudar a nossa Rede Neural.

columns_feat = [

    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

(iris_modelo.png)

In [ ]:
#Construindo a Rede Neural - DNNClassifier

classifier = tf.estimator.DNNClassifier(

    feature_columns=columns_feat,

    # 2 camadas ocultas com 10 nós cada
    hidden_units=[10, 10],

    # O modelo está classificando 3 classes
    n_classes=3)


In [ ]:
#Definindo função de treinamento

def train_function(inputs, outputs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), outputs))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()


#Treinar o Modelo

classifier.train(
    input_fn=lambda:train_function(train_x, train_y, 100),
    steps=1000)


In [ ]:
#Definindo função de avaliação
def evaluation_function(attributes, classes, batch_size):
    attributes=dict(attributes)
    if classes is None:
        inputs = attributes
    else:
        inputs = (attributes, classes)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

# Avaliando o classificador
eval_result = classifier.evaluate(
    input_fn=lambda:evaluation_function(test_x, test_y, 100))